<a href="https://colab.research.google.com/github/JairParra/NLPChallengeAI4Good/blob/master/tools/preprocess_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Imports
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk import sent_tokenize
import re
from google.colab import files

# Load results data from CSV
baseresultats = pd.read_csv("Baseresultats - Liste brute.csv")

# Get data for year 2018
data_2018 = baseresultats[["NO", "OBJECTIFS_2018", "RESULTATS_2018", "Type de résultat"]].dropna()

# Regexp to detect presense of digit as word in french
word_digits_regexp = re.compile(r'zéro|un|deux|trois|quatre|cinq|six|sept|huit|neuf|dix|onze|douze|treize|quatorze|quinze|seize|dix-sept|dix-huit|dix-neuf|vingt|vingtetun|vingt-deux|vingt-trois|vingt-quatre|vingt-cinq|vingt-six|vingt-sept|vingt-huit|vingt-neuf|trente|Trenteetun|Trente-deux|Trente-trois|Trente-quatre|Trente-cinq|Trente-six|Trente-sept|Trente-huit|Trente-neuf|quarante|quaranteetun|quarante-deux|quarante-trois|quarante-quatre|quarante-cinq|quarante-six|quarante-sept|quarante-huit|quarante-neuf|cinquante|cinquanteetun|cinquante-deux|cinquante-trois|cinquante-quatre|cinquante-cinq|cinquante-six|cinquante-sept|cinquante-huit|cinquante-neuf|soixante|soixanteetun|soixante-deux|soixante-trois|soixante-quatre|soixante-cinq|soixante-six|soixante-sept|soixante-huit|soixante-neuf|soixante-dix|soixante-et-onze|soixante-douze|soixante-treize|soixante-quatorze|soixante-quinze|soixante-seize|soixante-dix-sept|soixante-dix-huit|soixante-dix-neuf|quatre-vingts|quatre-vingt-un|quatre-vingt-deux|quatre-vingt-trois|quatre-vingt-quatre|quatre-vingt-cinq|quatre-vingt-six|quatre-vingt-sept|quatre-vingt-huit|quatre-vingt-neuf|quatre-vingt-dix|quatre-vingt-onze|quatre-vingt-douze|quatre-vingt-treize|quatre-vingt-quatorze|quatre-vingt-quinze|quatre-vingt-seize|quatre-vingt-dix-sept|quatre-vingt-dix-huit|quatre-vingt-dix-neuf|cent', re.IGNORECASE)

# Regexp to detect digit as digit
digit_regex = re.compile(r'\d+')

# Donwload punctuation
nltk.download('punkt')

# Result of split with other data, like organization ID, Type of result etc.
sentences = []
for row in data_2018.itertuples():
  # Replace line breaks with points
  text = row.RESULTATS_2018
  text = text.replace('\n', '. ').replace('\r', '. ')
  # Split text into sentences
  for sent in sent_tokenize(text, language='french'):
    # If we have Autre, there is definitely no quantified impact
    if row[4] == "Autre":
      quantified = "no"
    # Otherwise we do not know, so keep empty to fill in by human
    else:
      quantified = ""
    # Find digit as digit or as word
    if digit_regex.match(sent) or word_digits_regexp.match(sent):
      has_digit = "yes"
    else:
      has_digit = "no"
    # Append each sentence as a separate row to list
    sentences.append([row.NO, sent, row[4], has_digit, quantified])
# Save as CSV to disk    
pd.DataFrame(data=sentences, columns=["NO", "sentence_RESULTATS_2018", "Type de résultat", "Has digit", "Has impact"]).to_csv("sentences_classified.csv")
# Download the file
files.download("sentences_classified.csv")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
